### Optymalizacja - problem przeszukiwania

Lista skupia się na implementacji dwóch algorytmów z wykorzystaniem heurystyk i jednej metaheurystyki w problemie odnajdywania najkrótszych ścieżek

#### 1. Dijkstra 

Jest to algorytm zachłanny wybierający zawsze węzeł o najmniejszej wartości funkcji kosztu $f$. W tym przypadku wykorzystana została zmodyfikowana implementacja, która jest oparta na kolejce priorytetowej. Modyfikacja polega na tym, że 
1. Nie inicjalizujemy kolejki wszystkimi wierzchołkami, lecz tylko tymi początkowymi 
hm no wlasnie 
2. Nie przetrzymujemy listy odwiedzonych wierzchołków - może to powodować oddwiedzanie już wcześniej przebytych węzłów, ale w tym ??